<a href="https://colab.research.google.com/github/aaaksenova/RussianConstructicon_NLP_tools/blob/change/Definitions_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подсчет частотных слов и словосочетаний

Подсчет самых частотных слов и словосочетаний в определениях по семантическим типам конструкций. 

by Анна Аксенова (aksanna_a@mail.ru)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from pymystem3 import Mystem
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
from string import punctuation
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from collections import Counter
from pprint import pprint

russian_stopwords = stopwords.words("russian")

m = Mystem()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-12-08 18:23:08--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.242, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-12-08 18:23:08--  http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)... 5.45.220.12, 2a02:6b8:0:2002::13
Connecting to cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)|5.45.220.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1FU3qyx1VGtPZrMkww6C40lhryRKVVMatd2nOIo_ezVQ/edit?usp=sharing')

In [ ]:
sheet = wb.worksheet('Database for Radovan')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from operator import itemgetter

In [ ]:
df = pd.DataFrame(sheet.get_all_values())

In [ ]:
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
df['Definition in Russian']

1       Конструкция используется для того, чтобы напом...
2       Конструкция используется для выражения недоуме...
3       Конструкция обозначает, что по мнению говоряще...
4       Конструкция обозначает, что говорящий интересу...
5       Конструкция выражает сравнение и обозначает сх...
                              ...                        
2255    Конструкция используется, когда говорящий кате...
2256    Конструкция выражает запрет на [некоторое дейс...
2257    Конструкция используется, когда говорящий побу...
2258                                                     
2259                                                     
Name: Definition in Russian, Length: 2259, dtype: object

In [ ]:
import re

In [ ]:
line = 'Конструкция обозначает ситуацию, в которой у [участника]Experiencer нет необходимости или причины выполнять некоторое [действие]Action. Конструкция может также выражать запрет на совершение действия (пример 5). Ударение в "незачем" стоит на не-.'
re.sub(r'(\"|\')(.+?)(\"|\')', r'«\2»', line)

'Конструкция обозначает ситуацию, в которой у [участника]Experiencer нет необходимости или причины выполнять некоторое [действие]Action. Конструкция может также выражать запрет на совершение действия (пример 5). Ударение в «незачем» стоит на не-.'

In [ ]:
roles = []
for idx, i in df.iterrows():
    roles.append(set(re.findall(r'\[.+?\](\w+)', 
                                i['Definition in Russian'] + ' ' + i['Example 1']
                                + ' ' + i['Example 2']
                                + ' ' + i['Example 3']
                                + ' ' + i['Example 4']
                                + ' ' + i['Example 5'])))

In [ ]:
clean = []
for idx, i in df.iterrows():
    clean.append(re.sub(r'(\"|\')(.+?)(\"|\')', r'«\2»', i['Definition in Russian']))

In [ ]:
roles = [', '.join(list(i)) for i in roles]
df1 = pd.DataFrame({'roles' : roles, 'def' : clean}).to_csv('df.csv')

In [ ]:
all_labels = []
for i in roles:
    all_labels.extend(i)

In [ ]:
pprint(Counter(all_labels))

Counter({'Action': 470,
         'Theme': 450,
         'Situation': 212,
         'Participant': 179,
         'Agent': 80,
         'Property': 71,
         'Quantity': 63,
         'Evaluation': 62,
         'Experiencer': 60,
         'Event': 60,
         'Standard': 35,
         'Cause': 31,
         'Result': 31,
         'Set': 28,
         'State': 27,
         'Activity': 25,
         'Measure': 25,
         'Comparee': 24,
         'Location': 22,
         'Time': 21,
         'Possessor': 19,
         'Content': 18,
         'Speaker': 18,
         'Element': 18,
         'Condition': 15,
         'Addressee': 14,
         'Associated': 14,
         'Source': 13,
         'Patient': 11,
         'Parameter': 11,
         'Goal': 10,
         'Maleficiary': 10,
         'Instrument': 10,
         'Actant': 9,
         'Stimulus': 9,
         'Beneficiary': 9,
         'Landmark': 8,
         'Concession': 7,
         'Topic': 6,
         'Circumstance': 6,
         'Purpose'

In [ ]:
sheet.add_worksheet(rows=20,cols=2,title='runs')

In [ ]:
sem_types = list(set(df['SemType1']))

In [ ]:
punctuation = punctuation + ', ' + ' [' + '\n' + '. '

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


def TfidfFeatures(texts, topn=30, min_df=0.01, ngram_range=(1,3)):
    vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True,
                                 min_df=min_df, ngram_range=ngram_range,
                                 stop_words=set(list(punctuation) + russian_stopwords))
    collection = " ".join(texts) if type(texts) == list else texts
    X = vectorizer.fit_transform([collection])
    feature_names = vectorizer.get_feature_names()
    from_sorted=sort_coo(X.tocoo())[:topn]
    scores, features = [], []
    for idx, score in from_sorted:
        scores.append(round(score, 3))
        features.append(feature_names[idx])
    important_words = {features[idx]: scores[idx] for idx in range(len(features))}
    important_words = sorted(important_words.items(), key=itemgetter(1), reverse=True)
    return important_words

In [ ]:
def preprocess(sem_type):
    definitions = [re.sub(r'\[(.+?)\]\s?\s?\S+', r'\1', i) for i in sem_type]
    definitions = [[n for n in m.lemmatize(i) if n != ' ' and n not in punctuation] for i in definitions]
    texts = [' '.join(d) for d in definitions]
    return texts

In [ ]:
common_ngrams = pd.DataFrame()

In [ ]:
for sem_type in sem_types:
    texts = preprocess(df[df['SemType1']==sem_type]['Definition in Russian'])
    try:
        ngrams = TfidfFeatures(texts, topn=300, ngram_range=(2,4))
        common_ngrams[sem_type] = [i[0] for i in ngrams[:50]]
    except:
        print(sem_type)


Comitative
Request
Discourse structure
Root modality
SemType1
Mirative
Prohibition
Permission
Epistemic modality


In [ ]:
common_ngrams

,Threat,Manner,Degree of accuracy,Spatial expression,Degree of intensity,Cause,Instrument,Attitude,Salient property,Measure,Actionality,Pluractionality,Routine,Purpose,Assessment,Non-standard subject,Caritive,Concession,Time,Non-existence,Taxis,Calculation,Result,Quantification,Condition,Source of opinion,Comparison,Temporary characteristics,Timeline,Phase of action,Addressee,Reaction to the previous discourse,Possession
0,толкование база оставлять изменение,конструкция обозначать,конструкция использоваться,конструкция обозначать,некоторый объект,конструкция обозначать,конструкция обозначать,конструкция выражать,конструкция обозначать,конструкция обозначать,конструкция указывать,часть предложение,отношение говорить,участник ситуация прошлый совершать,толкование база,конструкция обозначать,конструкция обозначать,участник ситуация,отрицание обозначать,конструкция обозначать,часть предложение,конструкция использоваться,складываться ситуация,конструкция обозначать,конструкция обозначать,участник ситуация,тема действие,конструкция обозначать,иметь форма,участник долго,конструкция обозначать,конструкция использоваться,конструкция означать
1,толкование база оставлять,либо действие,число десяток сотня кроме,участник ситуация,субъект это,либо объект,свой результат помощь,негативный отношение,обозначать некоторый,конструкция использоваться,указывать ситуация распределениия объект,первый часть,негативный отношение говорить,участник ситуация прошлый,означать говорить,участник ситуация качество либо,причина предпосылка,толькование сайт оставлять изменение,конструкция двойной отрицание обозначать,участник ситуация,событие который,математический операция,полный отрицание,принятие решение,участник ситуация,участник ситуация субъект представлять,объект тема действие,обозначать некоторый,глагол иметь форма,конструкция обозначать,обозначать участник,конструкция выражать,объект конструкция означать
2,толкование база,участник ситуация,число десяток сотня,обозначать ситуация,пользование субъект,цель достигаться субъект помощь,свой результат,отношение говорить,конструкция обозначать некоторый,количество объект,указывать ситуация распределениия,являться фон событие действие,негативный отношение,участник ситуация,конструкция означать говорить,участник ситуация качество,действие ситуация,толькование сайт оставлять,конструкция двойной отрицание,обозначать участник,обозначать действие,конструкция использоваться выражение,отрицание либо возможный,подсчет количество,конструкция использоваться,участник ситуация субъект,объект тема,конструкция обозначать некоторый,глагол иметь,участник долго усиленно делать,конструкция обозначать участник,использоваться выражение,объект конструкция
3,оставлять изменение,ситуация конструкция,число десяток,обозначать место,подчеркивать длительный отсутствие некоторый,цель достигаться субъект,результат помощь,негативный отношение говорить,участник присваивать,стандартный единица,указывать ситуация,являться фон событие,конструкция выражать,ситуация прошлый совершать,конструкция означать,участник ситуация,являться достаточно очевидный факт,толькование сайт,конструкция двойной,обозначать действие,конструкция обозначать действие,использоваться выражение,отрицание либо,обозначать подсчет количество,событие действие,участник ситуация конструкция использоваться,человек инструмент конструкция обозначать,сторона конструкция обозначать,форма настоящий время либо,участник долго усиленно,совершать некоторый действие,конструкция использоваться выражение,означать говорить
4,конструкция обозначать,ситуация действие,числительное обычно выражать круглый,конструкция обозначать ситуация,подчеркивать длительный отсутствие,цель достигаться,обозначать либо,конструкция выражать негативный отношение,участник присваивать имя,числительное единица измеренение объект,указывать количество,являться фон,конструкция выражать крайне,ситуация прошлый,конструкция выражать,субъект организация поручение,являться достаточно очевидный,сайт оставлять изменение,двойно

In [ ]:
texts = preprocess(df['Definition in Russian'])
ngrams = TfidfFeatures(texts, topn=8000, ngram_range=(2,4))
ngrams[:50]

[('конструкция обозначать', 0.581),
 ('конструкция использоваться', 0.304),
 ('участник ситуация', 0.18),
 ('конструкция выражать', 0.143),
 ('использоваться выражение', 0.115),
 ('конструкция использоваться выражение', 0.096),
 ('действие конструкция', 0.096),
 ('обозначать участник', 0.093),
 ('объект конструкция', 0.087),
 ('конструкция обозначать участник', 0.081),
 ('либо действие', 0.075),
 ('оставлять изменение', 0.065),
 ('обозначать участник ситуация', 0.065),
 ('обозначать действие', 0.065),
 ('конструкция означать', 0.065),
 ('обозначать некоторый', 0.062),
 ('конструкция обозначать некоторый', 0.062),
 ('количество объект', 0.059),
 ('ситуация конструкция', 0.056),
 ('негативный отношение', 0.056),
 ('отношение говорить', 0.053),
 ('негативный отношение говорить', 0.053),
 ('конструкция обозначать участник ситуация', 0.053),
 ('конструкция обозначать действие', 0.053),
 ('действие ситуация', 0.05),
 ('действие который', 0.05),
 ('событие действие', 0.047),
 ('некоторый объе